In [1]:
!pip install xpinyin

In [2]:
from xpinyin import Pinyin
import pandas as pd
import numpy as np

In [3]:
import difflib
import Levenshtein

In [4]:
p = Pinyin()

In [5]:
# 导入英文单词
data = pd.read_csv("english.csv", index_col=0)

In [6]:
data.head(2)

,word,phonetic
0,"a, an",[ə；æn]
1,abandon,[əˈbændən]


In [7]:
yb2py ={" ":"","'":"","[":"","]":"",",":"", "/":"","ˈ":"","ˌ":"","ː":"",
        "i":"yi","ɪ":"yi","e":"ai","æ":"a","ə":"a","ʌ":"a","a":"a","ɑ":"a","ɔ":"ou","u":"u",
       "eɪ":"ei","aɪ":"ai","ɔɪ":"wei","əu":"ou","au":"ao","ɪə":"yi","ɛə":"aei","uə":'wei',
       "p":"p","b":"b","t":"t","d":"d","k":"k","ɡ":"g","f":"f","v":"u","s":"s","z":"s",
       "θ":"si","ð":"zhe","ʃ":"shi","ʒ":"yi","h":"he","r":"r","tʃ":"qi","dʒ":"ji",
       "tr":"que","dr":"jue","ts":"ci","dz":"zi","m":"m","n":"n","ŋ":"en","w":"wo","j":"","ɛ":"ai",
       "o":"o","ɝ":"er","ɜ":"e","ʊ":"o","ɚ":"er","ō":"o","l":""}

def yb_2_py(wordYB):
    '''Converts IPA symbols to Pinyin.'''
    py = ""
    index = 0
    YBLen = len(wordYB)
    while index<YBLen:
        if index+1<YBLen and wordYB[index:index+2] in yb2py:
            py += yb2py[wordYB[index:index+2]]
            index+=2
        else:
            py += yb2py[wordYB[index]]
            index+=1
    return py

In [8]:
yb_2_py("skaɪ")

'skai'

In [9]:
# 预处理
# 词
data.loc[0, "word"] = "an"
data.loc[992, "word"] = "an"
# 音标
phonetic_to_yb = lambda item: item.split(";")[0].split(")")[-1].replace(":","ː").strip()
ybs = data["phonetic"].apply(phonetic_to_yb)
ybs[0]='æn'
data = data.assign(ybs=ybs)
# 拼音
pys = data["ybs"].apply(yb_2_py)
data = data.assign(pys=pys)

data.head(2)

,word,phonetic,ybs,pys
0,an,[ə；æn],æn,an
1,abandon,[əˈbændən],[əˈbændən],abandan


In [10]:
# 单词筛选
data = data.loc[(data['word'].str.len()<=5) & (data['word'].str.len()>=3)]
data.reset_index(drop=True, inplace=True)
print(f"{len(data)} words remain.")

1196 words remain.


In [11]:
chinese = u"有备而来"
pinyin = p.get_pinyin(chinese).split("-")
str1 = ''.join(pinyin[1:3])
print(str1)

beier


In [12]:
# 转换音标
data = data.assign(
    diff_sim = data['word'].apply(lambda str2: difflib.SequenceMatcher(None, str1,str2).ratio())
)

In [13]:
print('Top 5 most similar words by `difflib.SequenceMatcher`:')
data.sort_values('diff_sim', ascending=False).head(5)

Top 5 most similar words by `difflib.SequenceMatcher`:


,word,phonetic,ybs,pys,diff_sim
83,beer,[bɪr],[bɪr],byir,0.888889
81,bee,[biː],[biː],byi,0.750000
82,beef,[biːf],[biːf],byif,0.666667
77,bear,[ber],[ber],bair,0.666667
98,bird,[bɜːrd],[bɜːrd],berd,0.666667
